## Prerequisites and installation instructions for Colab

If you want to run this notebook in Google Colab, run the following cell. Otherwise follow [installation instructions](https://github.com/BlueBrain/BlueGraph/blob/master/README.rst#installation) to install BlueGraph and its dependencies locally.

In [ ]:
# Install bluegraph
! git clone https://github.com/BlueBrain/BlueGraph
! cd BlueGraph && pip install .[cord19kg]

# Install graph-tool
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keys.openpgp.org --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool=2.37 python3-cairo python3-matplotlib

DATA_PATH = "BlueGraph/cord19kg/examples/data/"

## Prerequisites and installation instructions for a local environment

In order to run this notebook `graph-tool` must be installed manually (it cannot be installed as a part of `pip install bluegraph`, as it is not an ordinary Python library, but a wrapper around a C++ library). Please, see [graph-tool installation instructions](https://git.skewed.de/count0/graph-tool/-/wikis/installation-instructions#native-installation) (currently, BlueGraph supports :code:`graph-tool<=2.37`.)

We recommend using `conda` for installing `graph-tool`. For example:

```
conda install -c conda-forge graph-tool==2.37
```

or as a part of a new `conda` environment:

```
conda create --name <your_environment> -c conda-forge graph-tool==2.37
conda activate <your_environment>
```


BlueGraph and the set of dependecies supporting custom tools for CORD-19 analysis can be installed using:

 ```
 pip install bluegraph[cord19kg]
 ```

# Topic-centered co-occurrence network analysis of CORD-19

In this notebook we will perform interactive exploration and analysis of a topic-centered subset of the [CORD-19](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) dataset using the `cord19kg` package. The exploration and analysis techniques presented here focus on named entities and their co-occurrence in the scientific articles constituting the dataset.

The input data for this notebook contains the named entities extracted from the 3000 most relevant articles to the query _"Glucose is a risk factor of COVID-19"_ obtained using the article search model [BlueSearch](https://github.com/BlueBrain/Search). The entity extraction was performed using the Named Entity Recognition (NER) techniques also included in [BlueSearch](https://github.com/BlueBrain/Search). The entities represent 10 different types (i.e. proteins, chemicals, drugs, diseases, conditions, organs, organisms, pathways, cell types, cell compartments). 

The interactive literature exploration through the named entity co-occurrence analysis consisting of the following steps:

1. __Data preparation__ step converts raw mentions into aggregated entity occurrence statistics.
2. __Data curation__ step allows the user to manage extracted entities: modify, filter them and link to the ontology.
3. __Network generation__ step allows creating entity co-occurrence networks based on paper-, section- and paragraph-level co-occurrence relations between entities. These entity relations are quantified using mutual-information-based scores (pointwise mutual information and its normalized version).
4. __Network visualization and analysis__ step allows the user to perform interactive network visualization, edit network elements and perform its analysis (spanning tree, mutual-information based shortest paths between entities, etc).

In [26]:
import json
import os
import zipfile

import pandas as pd

import dash_cytoscape as cyto

from kgforge.core import KnowledgeGraphForge

from cord19kg.utils import (generate_curation_table,
                           link_ontology,
                           generate_cooccurrence_analysis,
                           download_from_nexus)
from bluegraph.backends.graph_tool import (GTMetricProcessor,
                                           GTPathFinder,
                                           GTGraphProcessor,
                                           GTCommunityDetector)
from cord19kg.apps.curation_app import curation_app
from cord19kg.apps.visualization_app import visualization_app

In [3]:
cyto.load_extra_layouts()

## 1. Data preparation

The input dataset contains occurrences of different terms in paragraphs of scientific articles from the CORD-19 dataset previously extracted by means of a NER model.
The dataset is stored in Blue Brain Nexus.

In [4]:
# Blue Brain Nexus bucket to download data from

nexus_bucket = "bbp/kg-inference"
nexus_endpoint = "https://bbp.epfl.ch/nexus/v1"
nexus_config_file = f"{DATA_PATH}../config/data-download-nexus.yml"


## Get an authentication token

For now, the [Nexus web application](https://bbp.epfl.ch/nexus/web) can be used to get a token. We are looking for other simpler alternatives.

- Step 1: From the opened web page, click on the login button on the right corner and follow the instructions.

![login-ui](./login-ui.png)

- Step 2: At the end you’ll see a token button on the right corner. Click on it to copy the token.

![login-ui](./copy-token.png)


In [10]:
import getpass

In [11]:
TOKEN = getpass.getpass()

In [11]:
from kgforge.core import KnowledgeGraphForge, Resource
from more_itertools import bucket

forge = KnowledgeGraphForge(nexus_config_file, bucket=nexus_bucket, endpoint=nexus_endpoint, token=TOKEN)


<action> _register_one
<succeeded> False
<error> RegistrationError: resource has invalid JSON-LD payload


## Download Data

In [20]:
data_version = "v1_13_10_2022"
try:
    print(f"Data path: '{DATA_PATH}'")
except NameError:
    DATA_PATH = "../data/"
    print(f"Data path: '{DATA_PATH}'")

Data path: '../data/'


In [22]:
%%time
ner_output_resource = forge.retrieve("https://bbp.epfl.ch/neurosciencegraph/data/2680c892-d1ae-4c60-8e1f-d917acb13ae8", version=data_version),
forge.download(ner_output_resource, "distribution.contentUrl", DATA_PATH)
print("Done.")

Done.
CPU times: user 2.14 s, sys: 1.35 s, total: 3.5 s
Wall time: 12.1 s


In [27]:
data = pd.read_csv(f"{DATA_PATH}/results_v1.csv")

In [28]:
data['occurrence'] = data['paper_id'].map(str) + ':_:' + data['paragraph_id'].map(str)
data=data[['entity', 'entity_type', 'occurrence', 'start', 'end']]

In [29]:
data.sample(5)

,entity,entity_type,occurrence,start,end
2871853,CMS,BRAIN_REGION,db5957b6d321fd2a6d571276ba0ec841:_:mneNiYMBVjZ...,360,363
1439426,alpha - synuclein,GENE,7b373a1e4591d557f5ceb5dd417de99c:_:eXWMiYMBVjZ...,1348,1363
1558918,somatostatin neurons,CELL_TYPE,75b35853ba820d1bd13abcea7d79dffc:_:D9GMiYMBxas...,439,459
2447352,cells,CELL_TYPE,c937823a2333971815bfee51b930abf4:_:BtOMiYMBxas...,55,60
874784,neurons,CELL_TYPE,4003af9ad8f2e905bac1288fb35ac71f:_:jnSLiYMBVjZ...,310,317


On the first preparation step, we group and aggregate the input data by unique entities.

In [30]:
%%time
print("Prepating curatation data...")
curation_input_table, factor_counts = generate_curation_table(data)
print("Done.")

Prepating curatation data...


/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/cord19kg/utils.py:325: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: x.split(":")[0])


Cleaning up the entities...
Aggregating occurrences of entities....
Done.
CPU times: user 24.5 s, sys: 635 ms, total: 25.1 s
Wall time: 25.3 s


The resulting dataframe contains a row per unique named entity together with the following occurrence data: 
- sets of unique paragraphs, papers, sections, where the corresponding entity is mentioned (`paper`, `section`, `paragraph` columns);
- number of total entity occurrences (the `raw_frequency` column);
- number of unique papers where it occurs (the `paper_frequency` column);
- unique entity types assigned by the NER model (the `entity_type` column, multiple types are possible).
- raw entity types assigned by the NER model with the multiplicity of thier occurrence (the `raw_entity_types` column).


In [31]:
curation_input_table.sample(5)

,entity,entity_type,paragraph,paper,section,paper_frequency,raw_entity_types,raw_frequency
31939,orl1,GENE,[f15b72f4486ff3b58fcdd0646e2b20fa:_:fdSNiYMBxa...,"[bb8a28b8f55b2bbff17be481184799e4, f15b72f4486...","[bb8a28b8f55b2bbff17be481184799e4:_, 96539fb63...",3,"[GENE, GENE, GENE, GENE, GENE, GENE, GENE, GEN...",10
2916,apaf -1,GENE,[1455a350a4536b423efa42ac1934fa2d:_:ls-LiYMBxa...,"[1a69b43e170f26e0dedc046963f6fb53, 6cc4886241d...","[8b596c883cb722ef30a1022a4281074f:_, 3ec5ed4ac...",33,"[GENE, GENE, GENE, GENE, GENE, GENE, GENE, GEN...",43
23371,lateral septal area,BRAIN_REGION,[53f7c7400b9a94cb7c07a63f5ad95857:_:03SLiYMBVj...,[53f7c7400b9a94cb7c07a63f5ad95857],[53f7c7400b9a94cb7c07a63f5ad95857:_],1,"[BRAIN_REGION, BRAIN_REGION, BRAIN_REGION]",3
2695,anterior ipn,BRAIN_REGION,[84a73066555148ee3c85f0d5d79a52c5:_:9HWMiYMBVj...,[84a73066555148ee3c85f0d5d79a52c5],[84a73066555148ee3c85f0d5d79a52c5:_],1,"[BRAIN_REGION, BRAIN_REGION, BRAIN_REGION, BRA...",4
26105,mesolimbic,"CELL_TYPE, BRAIN_REGION",[79c3b317a2e65a188b4b5b6e51142065:_:xtGMiYMBxa...,"[e5da6a86129e83b0bea1816622e44894, 76f7f9bf2ac...","[98fcac82b1a0b396b7db48409e085f4e:_, 6227e24c6...",292,"[BRAIN_REGION, BRAIN_REGION, BRAIN_REGION, BRA...",571


The second output of the data preparation step outputs the counts of different instances of occurrence factors: number of distinct papers/sections/paragraphs in the input corpus.

In [32]:
factor_counts

{'paper': 12292, 'section': 12292, 'paragraph': 344610}

## 2. Data curation

### Loading the NCIT ontology linking data (To be updated with BBP ontologies)

To group synonymical entities in the previously extracted table (e.g. `ace2`, `ace-2`, `angiotensin-converting enzyme 2`), as well as assign additional semantics to these entities (e.g. human-readable definition, taxonomy, etc), we peform further _linking_ of the entities to the terms from the [NCIT ontology](https://ncithesaurus.nci.nih.gov/ncitbrowser/).

To be able to perform such ontology linking, we load some additional (pre-computed using ML-based linking models) data.

In [35]:
%%time
print("Loading the ontology linking data...")

download_from_nexus(
    uri=f"{nexus_endpoint}/resources/covid19-kg/data/_/4fde1f8f-ee7f-435e-95a8-abb79139db93",
    output_path=DATA_PATH, config_file_path=nexus_config_file,
    nexus_endpoint=nexus_endpoint, nexus_bucket="covid19-kg/data", unzip=True)
print("\tLoading the linking dataframe in memory...")
ontology_linking = pd.read_csv(f"{DATA_PATH}/NCIT_ontology_linking_3000_papers.csv")


print("Loading ontology type mapping...")
ontology_linking_type_mapping_data = download_from_nexus(
    uri=f"{nexus_endpoint}/resources/covid19-kg/data/_/92bc2a04-6003-4f4d-85e1-dcc5f2352df2", 
    output_path=DATA_PATH, config_file_path=nexus_config_file,
    nexus_endpoint=nexus_endpoint, nexus_bucket="covid19-kg/data")
with open(f"{DATA_PATH}/{ontology_linking_type_mapping_data.distribution.name}", "rb") as f:
    type_mapping = json.load(f)
print("Done.")

Loading the ontology linking data...
Decompressing ...
	Loading the linking dataframe in memory...
Loading ontology type mapping...
Done.
CPU times: user 3.13 s, sys: 568 ms, total: 3.7 s
Wall time: 11.6 s


The ontology linking table contains the following columns:
- `mention` entity mentioned in the text
- `concept` ontology concept linked to the entity mention
- `uid` unique identifier of the ontology concept
- `definition` definition of the concept
- `taxonomy` taxonomy of semantic types associated with the concept

In [36]:
ontology_linking.sample(5)

,mention,concept,uid,definition,taxonomy
114583,human ku protein,hu protein,http://purl.obolibrary.org/obo/NCIT_C19434,A DNA-binding and thermostable low-molecular-w...,[('http://purl.obolibrary.org/obo/NCIT_C16319'...
42564,papular rash,"papulopustular rash, ctcae",http://purl.obolibrary.org/obo/NCIT_C143735,A disorder characterized by an eruption consis...,[('http://purl.obolibrary.org/obo/NCIT_C143171...
90444,krt7,krt5,NaN,NaN,NaN
24319,idp/isodesmosine,desmosine,http://purl.obolibrary.org/obo/NCIT_C124071,An amino acid derivative comprised of three al...,[('http://purl.obolibrary.org/obo/NCIT_C73539'...
133709,vitd insufficiency,renal failure,http://purl.obolibrary.org/obo/NCIT_C4376,An acute or chronic condition that is characte...,[('http://purl.obolibrary.org/obo/NCIT_C27600'...


### Interactive curation of  entity occurrence data

The package provides an interactive entity curation app that allows the user to visualize the entity occurrence data, modify it, perform ontology linking (see `Link to NCIT ontology` button), filter short or unfrequent entities.

The field `Keep` allows specifying a set of entities that must be kept in the dataset at all times (even if they don't satisfy the selected filtering criteria).

Finally the value specified in the `Generate Graphs from top N frequent entities` field corresponds to the number of top entities (by the frequency of their occurrence in papers) to be included in the co-occurrence network.

We load the prepared data table into the curation app as follows:

In [38]:
curation_app.set_table(curation_input_table.copy())

We can specify the default entities to keep.

In [39]:
default_entities_to_keep = []
curation_app.set_default_terms_to_include(default_entities_to_keep)

Finally, we set the ontology linking callback to be fired upon a click on the `Link to NCIT ontology` button.

In [40]:
curation_app.set_ontology_linking_callback(lambda x: link_ontology(ontology_linking, type_mapping, x))

### Launch the curation app

The application can be launched either inline (inside the current notebook) as below. Note that if you run this notebook in Colab, you may want to set a lower number of entities to include, in order to avoid long generation time ('Generate Graphs from top N frequent entities' field). Current default value is 200.

In [41]:
curation_app.run(port=8074, mode="external")

Opening port number 8074 failed: Address 'http://127.0.0.1:8074' already in use.
    Try passing a different port to run_server.. Trying port number 8075 ...
Dash app running on http://127.0.0.1:8075/


Exception in thread Thread-3:
Traceback (most recent call last):
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/Users/mfsy/anaconda3/envs/bluegraph/lib/python3.7/site-packages/retrying.py", line 200, in c

Or it can be opened externally (by the URL that you can open in a separate tab of your browser, try uncommenting, executing and doing Ctrl+Click on the displayed URL).

In [ ]:
# curation_app.run(port=8070, mode="external")

## 3. Co-occurrence network generation

Current curation table displayed in the curation app can be extracted using the `get_curated_table` method.

In [ ]:
curated_occurrence_data = curation_app.get_curated_table()
curated_occurrence_data.sample(3)

Before we can proceed we need to convert paper/section and paragraph columns into `set`.

In [ ]:
curated_occurrence_data["paper"] = curated_occurrence_data["paper"].apply(set)
curated_occurrence_data["paragraph"] = curated_occurrence_data["paragraph"].apply(set)
curated_occurrence_data["section"] = curated_occurrence_data["section"].apply(set)

We can also retreive current values of the `Keep` field (these entities will be also included in the resulting co-occurrence network).

In [ ]:
curation_app.get_terms_to_include()

In [ ]:
# Save curation table 
curated_occurrence_data.to_csv("/Users/mfsy/Documents/kg_from_ner/2022-10-12_v1/dke_processing/curated_linked_table.csv")

### Generating co-occurrence networks

In this section, we generate a paragraph-based entity co-occurrence network (i.e. an edge between a pair of entities is added if they co-occur in the same paragraph). Also some grapah and co-occurrence statistics are generated:

* computes co-occurrence statistics (such as frequency, pointwise mutual information and normalized pointwise mutual information) and assignes them as weights to the corresponding edges 

    * `ppmi`: given two entities (x, y), their _positive pointwise mutual information (PPMI)_ is defined as $PPMI(x, y) = \log_2{\frac{p(x, y)}{p(x)p(y)}} $, if $p(x) \neq 0$ and $p(y) \neq 0$, and $PPMI(x, y) = 0$ otherwise.

    * `npmi`: given two entities (x, y), their_normalized pointwise mutual information (NPMI)_ is defined as $NPMI(x, y) = \frac{PPMI(x, y)}{-\log_2{p(x, y)}} $.

* computes node centrality metrics (such as degree, RageRank) and them as weights to the nodes
* performs entity community detection based on different co-occurrence statistics
* computes mutual-information-based minimum spanning trees.

Before we run the co-occurrence analysis, we will create a dictionary with backend configurations for the analytics:

* we set metrics (centalities) computation to use `graph_tool`, 
* community detection to use `networkx` 
* and, finally, path search to use `graph_tool` as well.

In [ ]:
import time

In [ ]:
import time
backend_configs = {
    "metrics": "graph_tool",
    "communities": "networkx",
    "paths": "graph_tool"
}

In [ ]:
%%time
type_data = curated_occurrence_data[["entity_type"]].rename(columns={"entity_type": "type"})

graphs, trees = generate_cooccurrence_analysis(
    curated_occurrence_data,  factor_counts,
    n_most_frequent=curation_app.n_most_frequent,
    type_data=type_data, 
    factors=["paragraph"],
    keep=curation_app.get_terms_to_include(),
    cores=8,  # here set up the number of cores
    backend_configs=backend_configs)
print("Done.")

In [ ]:
graphs["paragraph"].edges(raw_frame=True).sample(5)

In [ ]:
graphs["paragraph"].nodes(raw_frame=True).sample(5)

#### Save or load co-occurence graph and spaning tree

In [ ]:
co_occurence_graph_nodes_path ="/Users/mfsy/Documents/kg_from_ner/2022-10-12_v1/dke_processing/co-occurence-graph-nodes.csv"
co_occurence_graph_edges_path ="/Users/mfsy/Documents/kg_from_ner/2022-10-12_v1/dke_processing/co-occurence-graph-edges.csv"

co_occurence_tree_nodes_path ="/Users/mfsy/Documents/kg_from_ner/2022-10-12_v1/dke_processing/co-occurence-tree-nodes.csv"
co_occurence_tree_edges_path ="/Users/mfsy/Documents/kg_from_ner/2022-10-12_v1/dke_processing/co-occurence-tree-edges.csv"


In [ ]:
# Save co-occurence graph
graphs["paragraph"].to_csv(co_occurence_graph_nodes_path, co_occurence_graph_edges_path)

In [ ]:
# Save spaning tree
trees["paragraph"].to_csv(co_occurence_tree_nodes_path, co_occurence_tree_edges_path)

In [ ]:
# Load saved co-occurence graph and spanning tree
graphs = {}
trees = {}
graphs["paragraph"]=PGFrame.from_csv(co_occurence_graph_nodes_path, co_occurence_tree_nodes_path)
graphs["paragraph"]=PGFrame.from_csv(co_occurence_graph_nodes_path, co_occurence_tree_edges_path)

## 4. Co-occurrence network visualization and analysis

### Programmatic analysis of the co-occurrence network 

#### Nearest neighours by co-occurrence scores

To illustrate the importance of computing mutual-information-based scores over raw frequencies, consider the following example, where we would like to estimate top closest (most related) neighbors to a specific term.

To do so, we will use the paragraph-based network and the raw co-occurrence frequency as the weight of our co-occurrence relation. The `top_neighbors` method of the `PathFinder` interface provided by the BlueGraph allows us to search for top neighbors with the highest edge weight. In this example, we use `graph_tool`-based `GTPathFinder` interface.

In [ ]:
paragraph_path_finder = GTPathFinder(graphs["paragraph"], directed=False)

In [ ]:
paragraph_path_finder.top_neighbors("hippocampus", 10, weight="frequency")

In [ ]:
paragraph_path_finder.top_neighbors("neuron", 10, weight="frequency")

We observe that 'glucose' and 'lung' share a lot of the closest neighbors by raw frequency. If we look into the list of top 10 entities by paragraph frequency in the entire corpus and we notice that 'glucose' and 'blood' co-occur the most with the terms that are simply the most frequent in our corpus, such as 'covid-19' and 'diabetes mellitus'.

(Closest inspection of the distribution of weighted node degrees suggests that the network contains _hubs_, nodes with significantly high-degree connectivity to other nodes.)

In [ ]:
graphs["paragraph"]._nodes

In [ ]:
graphs["paragraph"].nodes(raw_frame=True).nlargest(10, columns=["paragraph_frequency"])

To account for the presence of such hubs, we use the mutual-information-based scores presented above. They 'balance' the influence of the highly connected hub nodes such as 'covid-19' and 'diabetes mellitus' in our example.

In [ ]:
paragraph_path_finder.top_neighbors("hippocampus", 10, weight="npmi")

In [ ]:
paragraph_path_finder.top_neighbors("neuron", 10, weight="npmi")

#### IV. Graph metrics and centrality measures

BlueGraph provides the `MetricProcessor` interface for computing various graph statistics. As in the previous example, we will use `graph_tool`-based `GTMetricProcessor` interface.

In [ ]:
#paper_metrics = GTMetricProcessor(paper_network, directed=False)
paragraph_metrics = GTMetricProcessor(graphs["paragraph"], directed=False)

##### Graph density

Density of a graph is quantified by the proportion of all possible edges ($n(n-1) / 2$ for the undirected graph with $n$ nodes) that are realized.

In [ ]:
#print("Density of the paper-based network: ", paper_metrics.density())
print("Density of the paragraph-based network: ", paragraph_metrics.density())

The results above show that in the paragraph network, 39% of all possible term pairs co-occur at least once.

##### Node centrality (importance) measures

In this example we will compute the Degree and PageRank centralities only for the raw frequency, and the Betweenness centrality for the mutual-information-based scores. We will use methods provided by the `MetricProcessor` interface in the _write_ mode, i.e. computed metrics will be written as node properties of the underlying graph object.

_Degree centrality_ is given by the sum of weights of all incident edges of the given node and characterizes the importance of the node in the network in terms of its connectivity to other nodes (high degree = high connectivity).

In [ ]:
paragraph_metrics.degree_centrality("frequency", write=True, write_property="degree")

_PageRank centrality_ is another measure that estimated the importance of the given node in the network. Roughly speaking it can be interpreted as the probablity that having landed on a random node in the network we will jump to the given node (here the edge weights are taken into account").

https://en.wikipedia.org/wiki/PageRank

In [ ]:
paragraph_metrics.pagerank_centrality("frequency", write=True, write_property="pagerank")

We then compute the betweenness centrality based on the NPMI distances.

_Betweenness centrality_ is a node importance measure that estimates how often a shortest path between a pair of nodes will pass through the given node.

In [ ]:
paragraph_metrics.betweenness_centrality("distance_npmi", write=True, write_property="betweenness")

We can inspect the underlying graph object and observe the newly added properties:

In [ ]:
paragraph_metrics.graph.vp.keys()

Now, we will export this backend-specific graph object into a `PGFrame`.

In [ ]:
new_paragraph_network = paragraph_metrics.get_pgframe()

In [ ]:
new_paragraph_network.nodes(raw_frame=True).sample(5)

In [ ]:
print("Top 10 nodes by degree")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["degree"]).index:
    print("\t", n)

In [ ]:
print("Top 10 nodes by PageRank")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["pagerank"]).index:
    print("\t", n)

In [ ]:
print("Top 10 nodes by betweenness")
for n in new_paragraph_network.nodes(raw_frame=True).nlargest(10, columns=["betweenness"]).index:
    print("\t", n)

##### Compute multiple metrics in one go

Alternatively, we can compute all the metrics in one go. To do so, we need to specify edge attributes used for computing different metrics (if an empty list is specified as a weight list for a metric, computation of this metric is not performed). 

We select the paragraph-based network and re-compute all some of the previously illustrated metrics as follows:

In [ ]:
result_metrics = paragraph_metrics.compute_all_node_metrics(
    degree_weights=["frequency"],
    pagerank_weights=["frequency"],
    betweenness_weights=["distance_npmi"])

In [ ]:
result_metrics

### Interactive web-based analysis of the co-occurrence network 

#### Loading the generated graph into the visualization app

First of all, we set a backend for the visualization app (currently two backends are supported: based on `NetworkX` and `graph-tool`, in this example we use the latter).

In [ ]:
visualization_app.set_backend("graph_tool")

In [ ]:
# #  Run the following use NetworkX as the backend for the visualization app
# visualization_app.set_backend("networkx")

In [ ]:
visualization_app.add_graph(
    "Paragraph-based graph", graphs["paragraph"],
    tree=trees["paragraph"], default_top_n=100)

visualization_app.set_current_graph("Paragraph-based graph")

In [69]:
processor = visualization_app._graph_processor.from_graph_object(
        visualization_app._graphs["Paragraph-based graph"]["object"])
#processor.get_node("subcortical")

selected_node = "subcortical"
weights = {}
other_weights = {}
types= {}
for n in processor.neighbors(selected_node):
        other_weights[n] = {}
        weights[n] = processor.get_edge(selected_node, n)["npmi"]
        _node= processor.get_node(n)
        other_weights[n]["paragraph_frequency"] = _node["paragraph_frequency"]
        other_weights[n]["degree_frequency"] = _node["degree_frequency"]
        other_weights[n]["pagerank_frequency"] = _node["pagerank_frequency"]
        other_weights[n]["num_paper"] = len(_node["paper"])
        types[n] = _node["entity_type"] if "entity_type" in _node else None

#top_neighbors = top_n(weights, neighborlimit)
#print(weights)
print(other_weights)


{'subiculum': {'paragraph_frequency': 313.0, 'degree_frequency': 2634.0, 'pagerank_frequency': 0.0004512611514856408, 'num_paper': 6192}, 'substance p': {'paragraph_frequency': 209.0, 'degree_frequency': 1495.0, 'pagerank_frequency': 0.0003333954382972969, 'num_paper': 5796}, 'substantia nigra': {'paragraph_frequency': 2283.0, 'degree_frequency': 17603.0, 'pagerank_frequency': 0.0021544580781400063, 'num_paper': 45540}, 'subthalamic nucleus': {'paragraph_frequency': 640.0, 'degree_frequency': 4532.0, 'pagerank_frequency': 0.0006892442158339355, 'num_paper': 13032}, 'subventricular zone': {'paragraph_frequency': 457.0, 'degree_frequency': 4756.0, 'pagerank_frequency': 0.00068841177654795, 'num_paper': 12636}, 'superior colliculus': {'paragraph_frequency': 247.0, 'degree_frequency': 1898.0, 'pagerank_frequency': 0.000377471559904556, 'num_paper': 5436}, 'superior frontal gyrus': {'paragraph_frequency': 313.0, 'degree_frequency': 2054.0, 'pagerank_frequency': 0.00044341675935408886, 'num_

#### Loading papers' meta-data into the app

We now load an additional dataset containing some meta-data on the papers where the entities analyzed in this notebook occur.

In [ ]:
paper_medata = download_from_nexus(uri=f"{nexus_endpoint}/resources/{nexus_bucket}/_/8fc1e60c-1ebe-4173-82c0-9775a4917041",
                         output_path = DATA_PATH, config_file_path=nexus_config_file, nexus_endpoint=nexus_endpoint, nexus_bucket=nexus_bucket)
paper_data = pd.read_csv(f"{DATA_PATH}/{paper_medata.distribution.name}")
paper_data = paper_data.set_index("id")
paper_data.head(3)

We pass a callback for the lookup of paper meta-data to the visualization app using the `set_list_papers_callback` method.

In [ ]:
def list_papers(paper_data, selected_papers, limit=200):
    selected_paper_data = paper_data.loc[[int(p) for p in selected_papers]].head(200)
    return selected_paper_data.to_dict("records")

visualization_app.set_list_papers_callback(lambda x: list_papers(paper_data, x))

The ontology linking process described above is noisy, therefore, we would like to keep a possibility of accessing, the raw entities that were linked to particular ontology concepts. For this we define the function `get_aggregated_entities` that retreives such raw entities and we pass it to the visualization app using the `set_aggregated_entities_callback` method.

In [ ]:
def top_n(data_dict, n, smallest=False):
    """Return top `n` keys of the input dictionary by their value."""
    df = pd.DataFrame(dict(data_dict).items(), columns=["id", "value"])
    if smallest:
        df = df.nsmallest(n, columns=["value"])
    else:
        df = df.nlargest(n, columns=["value"])
    return(list(df["id"]))


def get_aggregated_entities(entity, n):
    if "aggregated_entities" in curated_occurrence_data.columns:
        aggregated = curated_occurrence_data.loc[entity]["aggregated_entities"]
    else:
        aggregated = [entity]
    if curation_input_table is not None:
        df = curation_input_table.set_index("entity")
        if entity in curated_occurrence_data.index:
            freqs = df.loc[aggregated]["paper_frequency"].to_dict()
        else:
            return {}
    else:
        df = data.copy()
        df["entity"] = data["entity"].apply(lambda x: x.lower())
        freqs = df[df["entity"].apply(lambda x: x.lower() in aggregated)].groupby("entity").aggregate(
            lambda x: len(x))["entity_type"].to_dict()
    if len(freqs) == 0:
        return {}
    return {e: freqs[e] for e in top_n(freqs, n)}

visualization_app.set_aggregated_entities_callback(
    lambda x: get_aggregated_entities(x, 10))

Finally, we create a dictionary `definitions` that will serve the visualization app as the lookup table for accessing the definitions of different ontology concepts.

In [ ]:
definitions = ontology_linking[["concept", "definition"]].groupby(
    "concept").aggregate(lambda x: list(x)[0]).to_dict()["definition"]
visualization_app.set_entity_definitons(definitions)

#### Launching the visualization app

As before, the interactive graph visualization app can be launched in two modes: inline and external. Here we recommend the external mode for better user experience.

In [ ]:
visualization_app.run(port=8081, mode="external")